In [82]:
import pymongo

In [83]:
class MongoDBConnection:
    def __init__(self, host, port, database, user=None, password=None, useSSL=False, additionalParams=None):
        self.connectiondict = {"host": host, "port": port, "database": database, "username": user,"password": password}
        self.host = host
        self.port = port
        self.database = database
        self.username = user
        self.password = password
        self.useSSL = useSSL
        if not additionalParams:
            self.additionalParams = []
        else:
            self.additionalParams = additionalParams
        self.dbconnection = self.connect()
        
    def connect(self):
        parameterappends = []

        if self.useSSL:
            ssl = "ssl=true"
            parameterappends.append(ssl)
            
        parameterappends.extend(self.additionalParams)
            
        param_string = "&".join(parameterappends)
        if param_string:
            param_string = '/?' + param_string
        
        if self.username and self.password:
            mongodbstring = "mongodb://" + self.username + ":" + self.password + "@" + self.host + ":" + str(self.port) + param_string
        else:
            mongodbstring = "mongodb://" + self.host + ":" + str(self.port) + param_string
        
        print(mongodbstring)

        
        dbconnection = pymongo.MongoClient(mongodbstring)
        db = dbconnection[self.database]
        
        return db
    
    def check_if_collection_exists(self, collectionName):
        if collectionName in self.dbconnection.list_collection_names():
            return True
        return False
    
    def select_collection(self, collectionName):
        return self.dbconnection[collectionName]
    
    def create_collection(self, collectionname, indexname=None):
        if self.check_if_collection_exists(collectionname):
            print("Collection \'{}\' already exists!".format(collectionname))
            return self.select_collection(collectionname)

        try:
            col = self.select_collection(collectionname)
            if indexname:
                col.create_index(indexname, unique=True)
            return col
        except Exception:
            print("Error during creation of Collection: {}".format(traceback.format_exc()))
            
    def insert(self, collectionname, data, indexkey=None):
        mongocollection = self.select_collection(collectionname)
        mongocollection.insert_many(data)
                
    def remove(self, collectionname, data):

        mongocollection = self.select_collection(collectionname)
        ids = []
        for datarow in data:
            ids.append(datarow["documentId"])
        deleted = mongocollection.delete_many({"documentId": {"$in": ids}})
        return deleted        

In [84]:
db = MongoDBConnection(host="localhost", port=27017, database="droppy")

mongodb://localhost:27017


In [85]:
db.check_if_collection_exists('wassim')

True

In [86]:
db.check_if_collection_exists('user_1')

False

In [87]:
db.create_collection("user_1")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'droppy'), 'user_1')

In [88]:
db.check_if_collection_exists('user_1') #We should populate the collection to become physically existant 

False

In [91]:
db.insert("user_1", [{'user_name':'user', 'consumption':0.11, 'date':'09/01/2024'}])

In [92]:
db.check_if_collection_exists('user_1') 

True